In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import sleep

In [3]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [5]:
d = {"Номер сделки":[], "Дата и время отправки заявки в банк":[], "Фамилия клиента":[], 
     "ФИО агента":[], "Компания":[], "Офис партнера":[], "Статус заявки":[], "Дата и время решения по заявке":[],
     "Мораторный отказ":[], "Срок отлагательного":[], "Дата последнего обновления статуса":[]}
result = []

In [ ]:
start = '2020-09-01'
stop = '2020-09-20'
link = 'https://beta-compas.domclick.ru/admin'               # тело ссылки
iter_link = link + '/deals/mortgage?createdAtGte=' \
+ start + '&createdAtLte=' + stop + '&offset='               # ссылка для итерации по страницам заявок
iterator = 24                                                # количество значений на странице (24 шт), последние символы в ссылке на странице с фильтром по заявкам
numerator = 0                                                # после фитра получит значение количества заявок в текущем фильтре
#https://beta-compas.domclick.ru/admin/deals/mortgage?createdAtGte=2020-09-01&createdAtLte=2020-09-17&offset=0

browser = webdriver.Chrome('chromedriver')
browser.get(link)
login = browser.find_element_by_id('login')
login.send_keys('login')
password = browser.find_element_by_name('password')
password.send_keys('password')
password.send_keys(Keys.ENTER)
# html = browser.page_source
sleep(5)
# browser.close()
browser.get(link + '/deals/mortgage?createdAtGte=' + start + '&createdAtLte=' \
            + stop + '&offset=' + str(0))
sleep(5)
black_count = browser.find_element_by_xpath('/html/body/div/div[4]/div/div/div/div/div/div[1]').text
numerator = int(black_count[18:(black_count.index('В')-1)])
# sleep(5)
if numerator % 24:
    count_iter = (numerator // 24) + 1
else:
    count_iter = numerator // 24                              # количество страниц фильтра

for i in range(count_iter):
    soup = BeautifulSoup(browser.page_source, "html.parser")
    new = soup.find('tbody').findAll('tr', 
                                     class_='el-table__row')  # тело таблицы заявок
    for j in new:
        string = j.findAll('div', class_="cell")              # всего 24 сроки и 11 значений в каждой
        for x in range(11):#len(string)):
            d.setdefault(list(d.keys())[x], []).append(string[x].text.strip())
    browser.get(iter_link + str(iterator*(i+1)))
    sleep(15)
browser.close()

In [ ]:
res = pd.DataFrame(d)
res.head()

In [52]:
res.shape

(463, 11)

In [54]:
res1 = res.drop_duplicates(subset=['Номер сделки'])

In [55]:
res1.shape

(453, 11)

In [56]:
res.to_excel("parse_compas.xlsx", index=False)